In [11]:
# import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

df_advanced = pd.read_csv('data/nba_2021_advanced_stats.csv', sep=';')[['Player', 'Code', 'Pos', 'PER']]
df_standard = pd.read_csv('data/nba_2021_standard_stats.csv', sep=';')[['Player', 'Code', 'G', 'MP', '3P%', 'PTS']]
df_salaries = pd.read_csv('data/nba_2022_salaries.csv', sep=';')[['Player', 'Code', '2021-22']].rename(columns={'2021-22': 'Salary'})

# join dfs
df = pd.merge(df_advanced, df_standard, how='inner', on=['Player', 'Code'])
df = pd.merge(df, df_salaries, how='inner', on=['Player', 'Code'])
df = df.drop_duplicates().reset_index(drop=True)

# We filter rows, keeping players that played at least ten matches, and played at least 5 minutes per game
df = df[(df['G'] >= 10) & (df['MP'] >= 5)]# We create an index column and filter Pos column
df['idx'] = df.index
df['Pos'] = df['Pos'].str.slice(stop=2)
df['Pos'] = df['Pos'].str.replace('-','')

In [13]:
# We create a dictionary that has information about the players, with a key indicating the index in df
# Also, we create a dictionary that has information on each player's position
players_dict = {}
positions_dict = {}

# We fill information on players and positions
for index, row in df.iterrows():
    players_dict[row['idx']] = {
        'name': row['Player'],
        'Pos': row['Pos'],
        'PER': row['PER'],
        '3P%': row['3P%'],
        'PTS': row['PTS'],
        'Salary': row['Salary']
    }
    pos = row['Pos']
    if pos not in positions_dict.keys():
        positions_dict[pos] = [row['idx']]
    else:
        positions_dict[pos].append(row['idx'])

In [21]:
# We create our variables and add it to a dictionary
def create_solver_and_player_assignment_variable(df):
    
    # We create the solver
    solver = pywraplp.Solver('simple_mip_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # We create the variables
    x_var_dict = {}
    for index, row in df.iterrows():
        x_var_dict[row['idx']] = solver.BoolVar(str('x_'+str(row['idx'])))
        
    return x_var_dict, solver

In [16]:
def total_players_constraint(solver, x_var_dict):
    ct = solver.Constraint(13, 15, 'TotalPlayers')
    for x in x_var_dict.keys():
        ct.SetCoefficient(x_var_dict[x], 1)
        
    return solver

In [17]:
def players_per_position(solver, x_var_dict, positions_dict):
    for position in positions_dict.keys():
        ct = solver.Constraint(2, 15, f'Players_Pos_{position}')
        for x in positions_dict[position]:
            ct.SetCoefficient(x_var_dict[x], 1)
            
    return solver

In [18]:
def total_salary(solver, x_var_dict, players_dict):
    ct = solver.Constraint(0, 112400000, 'TotalSalary')
    for x in x_var_dict.keys():
        ct.SetCoefficient(x_var_dict[x], players_dict[x]['Salary'])
        
    return solver

In [19]:
def three_points_shooters(solver, x_var_dict, players_dict):
    ct = solver.Constraint(4, 15, 'ThreePointsShooters')
    for x in x_var_dict.keys():
        if players_dict[x]['3P%'] > 0.4:
            ct.SetCoefficient(x_var_dict[x], 1)
            
    return solver

In [22]:
def add_constraints(df, players_dict, positions_dict, solver, add_three_point_shooters=False):
    solver = total_players_constraint(solver, x_var_dict)
    solver = players_per_position(solver, x_var_dict, positions_dict)
    solver = total_salary(solver, x_var_dict, players_dict)
    
    if add_three_point_shooters:
        solver = three_points_shooters(solver, x_var_dict, players_dict)
    return solver

In [20]:
def set_obj_function(solver, x_var_dict, players_dict):
    objective = solver.Objective()
    for x in x_var_dict.keys():
        objective.SetCoefficient(x_var_dict[x], players_dict[x]['PER'])
    objective.SetMaximization()
    solver.Solve()
    
    return solver, x_var_dict, objective

In [23]:
def get_team(x_var_dict, df):
    df_team = pd.DataFrame()
    # For each variable, we check if it has been already added 
    for idx in x_var_dict:
        if round(x_var_dict[idx].solution_value()) == 1:
            df_player = df[df['idx'] == idx]
            df_team = pd.concat([df_team, df_player], ignore_index=True)
    return df_team

In [24]:
# We run a first version, without constraining our number of three point shooters
x_var_dict, solver = create_solver_and_player_assignment_variable(df)
solver = add_constraints(df, players_dict, positions_dict, solver, add_three_point_shooters=False)
solver, x_var_dict, objective = set_obj_function(solver, x_var_dict, players_dict)
df_team = get_team(x_var_dict, df)

print(f"""
    Generated team total PER: {round(df_team.PER.sum(), 2)}. \n 
    Total Team's salary:  ${round(df_team.Salary.sum()/1000000, 2)} million""")
df_team


    Generated team total PER: 349.7. 
 
    Total Team's salary:  $111.79 million


,Player,Code,Pos,PER,G,MP,3P%,PTS,Salary,idx
0,Oshae Brissett,brissos01,SF,17.2,21,24.7,0.423,10.9,1701593.0,50
1,Dewayne Dedmon,dedmode01,C,24.5,16,13.1,0.200,7.1,5256308.0,92
2,Luka Dončić,doncilu01,PG,25.3,66,34.3,0.350,27.7,10174391.0,98
3,Daniel Gafford,gaffoda01,PF,22.2,54,14.6,NaN,7.0,1782621.0,117
4,Shai Gilgeous-Alexander,gilgesh01,SG,21.6,35,33.7,0.418,23.7,5495532.0,123
5,Alize Johnson,johnsal02,PF,23.5,18,10.5,0.167,5.2,1729217.0,184
6,Nikola Jokić,jokicni01,C,31.3,72,34.6,0.388,26.4,30510423.0,188
7,Enes Kanter,kanteen01,C,22.4,72,24.4,0.250,11.2,2641691.0,196
8,Zach LaVine,lavinza01,SG,21.5,58,35.1,0.419,27.4,19500000.0,205
9,Boban Marjanović,marjabo01,C,22.6,33,8.2,0.125,4.7,3500000.0,225
